# Particle Filter Test Suite

This notebook evaluates the statistical performance of Talyn's particle filters, including effective sample size, degeneracy, convergence, and resampling variability.


## 1. Track Effective Sample Size Over Time


In [ ]:
import numpy as np
N = 100
T = 20
particles = [0.0]*N
weights = np.ones(N)/N
ess_history = []
for t in range(T):
    particles = [p + np.random.normal(0,1) for p in particles]
    obs = np.random.normal(0,1)
    weights = np.exp(-0.5*(np.array(particles)-obs)**2)
    weights /= np.sum(weights)
    ess = 1.0 / np.sum(weights**2)
    ess_history.append(ess)
import matplotlib.pyplot as plt
plt.plot(ess_history)
plt.title('Effective Sample Size Over Time', fontsize=14)
plt.xlabel('Time', fontsize=12)
plt.ylabel('ESS', fontsize=12)
plt.show()


## 2. Visualize Particle Degeneracy


In [ ]:
plt.hist(weights, bins=30, alpha=0.7, color='teal')
plt.title('Particle Weights at Final Time', fontsize=14)
plt.xlabel('Weight', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.show()


## 3. Compare Filters Under Varying N


In [ ]:
Ns = [10, 50, 100, 500]
final_ess = []
for N in Ns:
    particles = [0.0]*N
    weights = np.ones(N)/N
    for t in range(10):
        particles = [p + np.random.normal(0,1) for p in particles]
        obs = np.random.normal(0,1)
        weights = np.exp(-0.5*(np.array(particles)-obs)**2)
        weights /= np.sum(weights)
    ess = 1.0 / np.sum(weights**2)
    final_ess.append(ess)
plt.plot(Ns, final_ess, 'o-')
plt.title('Final ESS vs N', fontsize=14)
plt.xlabel('N (Particles)', fontsize=12)
plt.ylabel('Final ESS', fontsize=12)
plt.xscale('log')
plt.show()


## 4. Show Convergence to True State


In [ ]:
true_state = 0.0
estimates = []
particles = [true_state]*N
weights = np.ones(N)/N
for t in range(T):
    particles = [p + np.random.normal(0,1) for p in particles]
    obs = np.random.normal(0,1)
    weights = np.exp(-0.5*(np.array(particles)-obs)**2)
    weights /= np.sum(weights)
    estimate = np.sum(np.array(particles)*weights)
    estimates.append(estimate)
plt.plot(estimates, label='Particle Filter Estimate')
plt.axhline(true_state, color='red', linestyle='--', label='True State')
plt.title('Convergence to True State', fontsize=14)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Estimate', fontsize=12)
plt.legend()
plt.show()


## 5. Plot Resampling Variability


In [ ]:
def multinomial_resample(particles, weights):
    idx = np.random.choice(len(particles), size=len(particles), p=weights)
    return [particles[i] for i in idx]
particles = [0.0]*N
weights = np.ones(N)/N
for t in range(T):
    particles = [p + np.random.normal(0,1) for p in particles]
    obs = np.random.normal(0,1)
    weights = np.exp(-0.5*(np.array(particles)-obs)**2)
    weights /= np.sum(weights)
    if 1.0/np.sum(weights**2) < N/2:
        particles = multinomial_resample(particles, weights)
plt.hist(particles, bins=30, alpha=0.7, color='purple')
plt.title('Particle Distribution After Resampling', fontsize=14)
plt.xlabel('State', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.show()
